In [1]:
import sys
print(sys.executable)

C:\Users\ssemedo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe


In [ ]:
import pandas as pd

# Lê o ficheiro CSV
df = pd.read_csv(r'C:\Users\ssemedo\OneDrive - The Summer Berry Company\Desktop\github\greends-ipython25_26\assignment_3\kiva_loans.csv')

# Verifica se carregou corretamente
df.head()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671205 entries, 0 to 671204
Data columns (total 20 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  671205 non-null  int64  
 1   funded_amount       671205 non-null  float64
 2   loan_amount         671205 non-null  float64
 3   activity            671205 non-null  object 
 4   sector              671205 non-null  object 
 5   use                 666973 non-null  object 
 6   country_code        671197 non-null  object 
 7   country             671205 non-null  object 
 8   region              614405 non-null  object 
 9   currency            671205 non-null  object 
 10  partner_id          657698 non-null  float64
 11  posted_time         671205 non-null  object 
 12  disbursed_time      668809 non-null  object 
 13  funded_time         622874 non-null  object 
 14  term_in_months      671205 non-null  float64
 15  lender_count        671205 non-nul

In [18]:
df.columns

Index(['id', 'funded_amount', 'loan_amount', 'activity', 'sector', 'use',
       'country_code', 'country', 'region', 'currency', 'partner_id',
       'posted_time', 'disbursed_time', 'funded_time', 'term_in_months',
       'lender_count', 'tags', 'borrower_genders', 'repayment_interval',
       'date', 'loan_usd'],
      dtype='object')

* Os valores dos empréstimos estão em várias moedas diferentes.

* Para comparar valores de forma justa entre países, precisamos de converter todos os empréstimos para uma moeda comum, escolhida como USD.

* O dicionário exchange_rates contém taxas de conversão para cada moeda presente no dataset.

In [10]:


# Dicionário de taxas de câmbio para USD
exchange_rates = {
    'USD': 1,
    'EUR': 1.1,
    'GBP': 1.25,
    'KES': 0.007,
    'PHP': 0.018,
    'UGX': 0.00027,
    'PEN': 0.27,
    'BOB': 0.14,
    # adicionar mais se necessário
}

# Criar coluna de valor do empréstimo convertido para USD
df['loan_usd'] = df.apply(
    lambda x: x['loan_amount'] * exchange_rates.get(x['currency'], 1),
    axis=1
)




* Cada empréstimo é multiplicado pela taxa correspondente para obter o seu valor em USD.

* Criar esta coluna permite:

1. Comparar empréstimos entre países de forma consistente
2. Facilitar cálculos de soma, média e outras métricas
3. Evitar confusões de moedas diferentes nas análises subsequentes

* O uso de apply + lambda percorre cada linha do dataset e aplica a conversão de forma eficiente.

# 1. Quais são os 10 países com maior valor total emprestado em USD?
* Objetivo: Identificar quais países recebem mais financiamento.
* Groupby
* Agregação (sum)
* Sort (sort_values)

In [20]:
df.groupby('country')['loan_usd'] \
  .sum() \
  .sort_values(ascending=False) \
  .head(10)


country
United States    31496375.0
Paraguay         29629075.0
El Salvador      26603650.0
Cambodia         19872700.0
Rwanda           16646300.0
Tajikistan       15030350.0
Ecuador          15000275.0
Vietnam          14891450.0
Colombia         14415600.0
Pakistan         13294750.0
Name: loan_usd, dtype: float64

# 2. Qual é a média de empréstimo por setor para empréstimos em USD?
* Objetivo: Comparar qual setor recebe mais dinheiro
* groupby
* sum()
* sort_values ()

In [24]:
df.groupby('sector')['loan_usd'] \
  .sum() \
  .sort_values(ascending=False)



sector
Agriculture       1.036811e+08
Food              8.758381e+07
Retail            6.562565e+07
Services          4.127288e+07
Clothing          3.069148e+07
Education         2.663288e+07
Housing           2.034486e+07
Personal Use      1.354111e+07
Arts              1.014058e+07
Health            8.517774e+06
Transportation    7.272309e+06
Construction      5.281801e+06
Manufacturing     4.303563e+06
Entertainment     1.242987e+06
Wholesale         8.154248e+05
Name: loan_usd, dtype: float64

# 3. Quais as atividades mais financiados (em usd)?
* Objetivo: ver quais as atividades que recebem mais dinheiro

In [32]:
df.groupby('activity')['loan_usd'] \
  .sum() \
  .sort_values(ascending=False) \
  .head(10)




activity
Farming                      3.299068e+07
Agriculture                  2.184100e+07
Food Production/Sales        2.074498e+07
Retail                       2.033119e+07
Clothing Sales               1.962099e+07
General Store                1.938558e+07
Personal Housing Expenses    1.935275e+07
Higher education costs       1.765414e+07
Livestock                    1.361504e+07
Services                     1.227503e+07
Name: loan_usd, dtype: float64

**Resultado**: Farming e agriculture lidera como sendo as atividades que recebem mais financiamento.

# 4. Quais os setores com maior numero médio de lenders por emprestimo

In [31]:
df.groupby('sector')['lender_count'] \
  .mean() \
  .sort_values(ascending=False)



sector
Wholesale         43.413249
Entertainment     31.762651
Education         29.335698
Health            29.086414
Construction      27.656031
Arts              27.055721
Manufacturing     24.833119
Clothing          24.674852
Services          24.393642
Agriculture       21.123881
Food              19.863410
Housing           19.266520
Retail            17.340498
Transportation    17.087833
Personal Use      11.789639
Name: lender_count, dtype: float64

**RESULTADO**: número médio de lenders por emprestimo é mais elevado no setor de WHOLESALE.
Há setores que atraem mais contribuidores do que outros.

# 5. Quais os paises onde cada lender contribui mais?
* Objetivo: criar nova variável
* Apply
* Lambda
* loan_per_lender

In [33]:
df['loan_per_lender'] = df.apply(
    lambda row: row['loan_usd'] / row['lender_count']
    if row['lender_count'] > 0 else None,
    axis=1
)


In [34]:
df.groupby('country')['loan_per_lender'] \
  .mean() \
  .sort_values(ascending=False) \
  .head(10)


country
Guam                                    716.666667
Paraguay                                330.352434
United States                           283.243816
Congo                                   252.172588
The Democratic Republic of the Congo    223.055440
Puerto Rico                             164.401544
Chile                                   102.434114
Mexico                                   89.743265
Rwanda                                   85.742197
Mali                                     76.079602
Name: loan_per_lender, dtype: float64

Contribuição média por lender no Guam ~716 USD

# 6. Qual a média de emprestimos das atividades agrícolas (em USD)?
* Objetivo: selecionar "Farm" e calcular medias de emprestimos em USD.
* Filtrar atividades que têm a palavra Farm

In [35]:
agri = df[df['activity'].str.contains('Farm', case=False, na=False)]




In [36]:
agri['loan_usd'].mean()


np.float64(464.4393378413339)

Resultado: ~464,44 USD por emprestimo agricola